In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt


# Load your data
data = pd.read_csv('new_data2.csv')
data = data.drop(['date(dd/mm/yy)'], axis=1)

# Handle missing values
data.fillna(data.mean(), inplace=True)

# Define the target variable
target = 'HYB'
X = data.drop(columns=[target])
y = data[target]

# Feature scaling
scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler.fit_transform(X)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Reshape input to be 3D for LSTM: [samples, time steps, features]
# Assuming a time step of 1 (you can modify based on your data needs)
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Build the bidirectional RNN model using LSTM layers
model = Sequential()
model.add(Bidirectional(LSTM(50, return_sequences=True, input_shape=(1, X_train.shape[2]))))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(50)))
model.add(Dropout(0.2))
model.add(Dense(1))  # Output layer for regression

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R-squared Score: {r2}")

# Plotting the results
plt.figure(figsize=(10, 6))
plt.plot(y_test.values, label='True')
plt.plot(y_pred, label='Predicted')
plt.title('True vs Predicted Values')
plt.legend()
plt.show()


ValueError: could not convert string to float: '06-08-2023'